In [12]:
import torch
from torch.autograd import Variable
import ResNetFeat
import yaml
import data
import os
import argparse
import numpy as np
import h5py
import json

In [13]:
with open('base_classes.json') as f:
    base_classes=json.load(f)

with open('novel_classes.json') as f:
    novel_classes=json.load(f)

In [14]:
cfg='train_save_data.yaml'
modelfile='checkpoints/ResNet10_sgm/19.tar'
model='ResNet10'
num_classes=10378
batch_size=16
maxiters=1000

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='Save features')
    parser.add_argument('--cfg', required=True, help='yaml file containing config for data')
    parser.add_argument('--modelfile', required=True, help='model file')
    parser.add_argument('--model', type=str, default='ResNet10', help='model')
    parser.add_argument('--num_classes', type=int,default=10378)
    parser.add_argument('--batchsize', default=16, type=int)
    parser.add_argument('--maxiters', default=1000, type=int)
    return parser.parse_args()

In [7]:
def get_features(model,data_loader):
    
    feature_set=[]
    label_set=[]
    for i, (x,y) in enumerate(data_loader):
        # ignoring the data that belong to base class
        index=0
        while True:
            if(y[index] not in novel_classes):
                y=torch.cat([y[0:index], y[index+1:]])
                x=torch.cat([x[0:index], x[index+1:]])
                index-=1
            index+=1

            if(len(y)==index):
                break

        if(len(y)==0):
            continue
            
        
        if i%10 == 0:
            print('{:d}/{:d}'.format(i, len(data_loader)))
        x = x.cuda()
        x_var = Variable(x)
        scores, feats = model(x_var)
        feature_set.extend(feats.data.cpu().numpy())
        label_set.extend(y)
        
        return (feature_set,label_set)
        
    

In [ ]:
def training_loop(features, labels,num_classes, params, batchsize=1000, maxiters=1000):
    featdim = features[0].shape[1]
    model = nn.Linear(featdim, num_classes)
    model = model.cuda()
    optimizer = torch.optim.SGD(model.parameters(), params.lr, momentum=params.momentum, dampening=params.momentum, weight_decay=params.wd)

    loss_function = nn.CrossEntropyLoss()
    loss_function = loss_function.cuda()
    for i in range(maxiters):
        (x,y) = lowshot_dataset.get_sample(batchsize)
        optimizer.zero_grad()

        x = Variable(x.cuda())
        y = Variable(y.cuda())
        scores = model(x)

        loss = loss_function(scores,y)
        loss.backward()
        optimizer.step()
        if (i%100==0):
            print('{:d}: {:f}'.format(i, loss.data[0]))

    return model

In [16]:
if __name__ == '__main__':
    with open(cfg,'r') as f:
        data_params = yaml.load(f)

    data_loader = data.get_data_loader(data_params)
    model = get_model(model, num_classes)
    model = model.cuda()
    model = torch.nn.DataParallel(model)
    tmp = torch.load(modelfile)
    if ('module.classifier.bias' not in model.state_dict().keys()) and ('module.classifier.bias' in tmp['state'].keys()):
        tmp['state'].pop('module.classifier.bias')
    
    model.load_state_dict(tmp['state'])
    model.eval()
    
#     feature_set,label_set=get_features(model,data_loader)
#     model = training_loop(feature_set,labels, numclasses, params, batchsize, maxiters)
    